# Introduction

# Load the data

You can download the data from Kaggle:https://www.kaggle.com/c/home-credit-default-risk

Since there are so many features in each tables, I made these data summaries for your quick check:

- Main table: http://nbviewer.jupyter.org/github/gukun770/Home-Credit/blob/master/data_summary_application_train.ipynb
- Other tables: http://nbviewer.jupyter.org/github/gukun770/Home-Credit/blob/master/data_summary_others.ipynb

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as scs
import collections

%matplotlib inline
plt.style.use('ggplot') # overall 'ggplot' style

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina') 

/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, auc, roc_curve, roc_auc_score
from sklearn.preprocessing import PolynomialFeatures, Imputer, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from operator import itemgetter
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

In [4]:
pd.set_option("display.max_colwidth",999)
pd.set_option("display.max_rows",999)
pd.set_option("display.max_columns",999)

Helper function ```col_name(name, df_description)```--> gives description of the column name defined in the table HomeCredit_columns_description.csv

In [5]:
col_description = pd.read_csv('data/HomeCredit_columns_description.csv',
                              index_col=0,encoding='cp1252')

def col_name(name, col_description=col_description):
    values = col_description[col_description['Row'] == name]['Description'].values
    print(name + ': \n')
    for i in values:
        print (i+'\n')
    table = col_description[col_description['Row'] == name]['Table'].values
    print('Table: {}\n'.format(table))

In [6]:
col_name('MONTHS_BALANCE')

MONTHS_BALANCE: 

Month of balance relative to application date (-1 means the freshest balance date)

Month of balance relative to application date (-1 means the information to the freshest monthly snapshot, 0 means the information at application - often it will be the same as -1 as many banks are not updating the information to Credit Bureau regularly )

Month of balance relative to application date (-1 means the freshest balance date)

Table: ['bureau_balance.csv' 'POS_CASH_balance.csv' 'credit_card_balance.csv']



In [7]:
ls data/

HomeCredit_columns_description.csv  bureau_balance.csv.zip
POS_CASH_balance.csv.zip            credit_card_balance.csv.zip
application_test.csv.zip            installments_payments.csv.zip
application_train.csv.zip           previous_application.csv.zip
bureau.csv.zip                      sample_submission.csv.zip


In [8]:
application_train = pd.read_csv('data/application_train.csv.zip') 
application_test = pd.read_csv('data/application_test.csv.zip') 

In [9]:
df_train = application_train.copy()
df_test = application_test.copy()

In [12]:
bureau = pd.read_csv('data/bureau.csv.zip')
POS = pd.read_csv('data/POS_CASH_balance.csv.zip')
previous_application = pd.read_csv('data/previous_application.csv.zip')
installments = pd.read_csv('data/installments_payments.csv.zip') 
ccb = pd.read_csv('data/credit_card_balance.csv.zip')
bureau_balance = pd.read_csv('data/bureau_balance.csv.zip')

In [13]:
df_previous = previous_application.groupby('SK_ID_CURR').median()
df_previous.rename(columns={'AMT_GOODS_PRICE':'previous_AMT_GOODS_PRICE','AMT_CREDIT':'previous_AMT_CREDIT',
                            'AMT_ANNUITY':'previous_AMT_ANNUITY'}, inplace=True)
df_train = pd.merge(df_train, df_previous, how='left', left_on='SK_ID_CURR', right_index=True)
df_test = pd.merge(df_test, df_previous, how='left', left_on='SK_ID_CURR', right_index=True)

In [14]:
df_pos = POS.groupby('SK_ID_CURR').mean()
df_train = pd.merge(df_train, df_pos, how='left', left_on='SK_ID_CURR', right_index=True)
df_test = pd.merge(df_test, df_pos, how='left', left_on='SK_ID_CURR', right_index=True)

In [ ]:
# status1 = bureau_balance[bureau_balance.STATUS=='1']
# status2 = bureau_balance[bureau_balance.STATUS=='2']
# status3 = bureau_balance[bureau_balance.STATUS=='3']
# status4 = bureau_balance[bureau_balance.STATUS=='4']
# status0 = bureau_balance[bureau_balance.STATUS=='0']
# statusX = bureau_balance[bureau_balance.STATUS=='X']
# statusC = bureau_balance[bureau_balance.STATUS=='C']

# status1.rename(columns={'STATUS':'STATUS1_CNT'},inplace=True)
# status2.rename(columns={'STATUS':'STATUS2_CNT'},inplace=True)
# status3.rename(columns={'STATUS':'STATUS3_CNT'},inplace=True)
# status4.rename(columns={'STATUS':'STATUS4_CNT'},inplace=True)
# status0.rename(columns={'STATUS':'STATUS0_CNT'},inplace=True)
# statusX.rename(columns={'STATUS':'STATUSX_CNT'},inplace=True)
# statusC.rename(columns={'STATUS':'STATUSC_CNT'},inplace=True)

# s1 = status1.groupby('SK_ID_BUREAU')[['STATUS1_CNT']].count()
# s2 = status2.groupby('SK_ID_BUREAU')[['STATUS2_CNT']].count()
# s3 = status3.groupby('SK_ID_BUREAU')[['STATUS3_CNT']].count()
# s4 = status4.groupby('SK_ID_BUREAU')[['STATUS4_CNT']].count()
# s0 = status0.groupby('SK_ID_BUREAU')[['STATUS0_CNT']].count()
# sC = statusC.groupby('SK_ID_BUREAU')[['STATUSC_CNT']].count()
# sX = statusX.groupby('SK_ID_BUREAU')[['STATUSX_CNT']].count()

# bureau = pd.merge(bureau, s1, how='left', left_on='SK_ID_BUREAU', right_index=True)
# bureau = pd.merge(bureau, s2, how='left', left_on='SK_ID_BUREAU', right_index=True)
# bureau = pd.merge(bureau, s3, how='left', left_on='SK_ID_BUREAU', right_index=True)
# bureau = pd.merge(bureau, s4, how='left', left_on='SK_ID_BUREAU', right_index=True)
# bureau = pd.merge(bureau, s0, how='left', left_on='SK_ID_BUREAU', right_index=True)
# bureau = pd.merge(bureau, sX, how='left', left_on='SK_ID_BUREAU', right_index=True)
# bureau = pd.merge(bureau, sC, how='left', left_on='SK_ID_BUREAU', right_index=True)

# bureau.loc[:,['STATUS1_CNT','STATUS2_CNT','STATUS3_CNT','STATUS4_CNT','STATUS0_CNT','STATUSX_CNT','STATUSC_CNT']] = bureau.loc[
#     :,['STATUS1_CNT','STATUS2_CNT','STATUS3_CNT','STATUS4_CNT','STATUS0_CNT','STATUSX_CNT','STATUSC_CNT']].fillna(0)

In [15]:
df_bureau = bureau.groupby('SK_ID_CURR').agg({
                                  'AMT_CREDIT_SUM':'sum','AMT_CREDIT_SUM_OVERDUE':'sum',
                                  'DAYS_CREDIT_UPDATE':'max','AMT_ANNUITY':'sum',
#                                 'STATUS1_CNT':'mean','STATUS2_CNT':'mean','STATUS3_CNT':'mean',
#                                 'STATUS4_CNT':'mean','STATUS0_CNT':'mean','STATUSX_CNT':'mean',
#                                 'STATUSC_CNT':'mean',

})

columns=[('AMT_CREDIT_SUM','BU_sum_AMT_CREDIT_SUM'),
         ('AMT_CREDIT_SUM_OVERDUE','BU_sum_AMT_CREDIT_SUM_OVERDUE'),
         ('DAYS_CREDIT_UPDATE','BU_max_DAYS_CREDIT_UPDATE'),
         ('AMT_ANNUITY','BU_sum_AMT_ANNUITY'),
#          ('STATUS1_CNT','BU_STATUS1_CNT'),
#          ('STATUS2_CNT','BU_STATUS2_CNT'),
#          ('STATUS3_CNT','BU_STATUS3_CNT'),
#          ('STATUS4_CNT','BU_STATUS4_CNT'),
#          ('STATUS0_CNT','BU_STATUS0_CNT'),
#          ('STATUSC_CNT','BU_STATUSC_CNT'),
#          ('STATUSX_CNT','BU_STATUSX_CNT'),
        ]

# df_bureau = bureau.groupby('SK_ID_CURR').mean()
df_bureau.columns=pd.MultiIndex.from_tuples(columns)
df_bureau.columns = df_bureau.columns.droplevel()
df_train = pd.merge(df_train, df_bureau, how='left', left_on='SK_ID_CURR', right_index=True)
df_test = pd.merge(df_test, df_bureau, how='left', left_on='SK_ID_CURR', right_index=True)

In [16]:
installments['payment_day_missed']=(installments.DAYS_ENTRY_PAYMENT>installments.DAYS_INSTALMENT)*1
installments['payment_day_missed_amt']=installments.DAYS_ENTRY_PAYMENT-installments.DAYS_INSTALMENT

installments['payment_missed']=(installments.AMT_INSTALMENT>installments.AMT_PAYMENT)*1
installments['payment_missed_amt']=installments.AMT_INSTALMENT-installments.AMT_PAYMENT

df_installments = installments.groupby('SK_ID_CURR').agg({'payment_day_missed':['sum','mean'],
                                  'payment_day_missed_amt':['mean','min'],'payment_missed':['sum','mean'],
                                   'payment_missed_amt':['mean','max'],                       
                                  'NUM_INSTALMENT_NUMBER':'mean'})
columns=[('payment_day_missed','payment_day_missed_sum'),('payment_day_missed','payment_day_missed_mean'),
         ('payment_day_missed_amt','payment_day_missed_amt_mean'),('payment_day_missed_amt','payment_day_missed_amt_max'),
         ('payment_missed','payment_missed_sum'),('payment_missed','payment_missed_mean'),
         ('payment_missed_amt','payment_missed_amt_mean'),('payment_missed_amt','payment_missed_amt_max'),
         ('NUM_INSTALMENT_NUMBER','NUM_INSTALMENT_NUMBER_mean')]
df_installments.columns=pd.MultiIndex.from_tuples(columns)
df_installments.columns = df_installments.columns.droplevel()
df_train = pd.merge(df_train, df_installments, how='left', left_on='SK_ID_CURR', right_index=True)
df_test = pd.merge(df_test, df_installments, how='left', left_on='SK_ID_CURR', right_index=True)

In [17]:
ccb['pct_balance_limit'] = (ccb['AMT_BALANCE']+1)/(ccb['AMT_CREDIT_LIMIT_ACTUAL']+1)
ccb['pct_AMT_INST_TOTAL'] = (ccb['AMT_INST_MIN_REGULARITY']+1)/(ccb['AMT_PAYMENT_TOTAL_CURRENT']+1)
df_ccb = ccb.groupby('SK_ID_CURR').mean()
df_ccb.rename(columns={'MONTHS_BALANCE':'ccb_MONTHS_BALANCE','SK_DPD_DEF':'ccb_SK_DPD_DEF','SK_DPD':'ccb_SK_DPD'}, inplace=True)
df_train = pd.merge(df_train, df_ccb, how='left', left_on='SK_ID_CURR', right_index=True)
df_test = pd.merge(df_test, df_ccb, how='left', left_on='SK_ID_CURR', right_index=True)

In [18]:
df_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START_x,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,SK_ID_PREV_x,previous_AMT_ANNUITY,AMT_APPLICATION,previous_AMT_CREDIT,AMT_DOWN_PAYMENT,previous_AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START_y,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,SK_ID_PREV_y,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,BU_sum_AMT_CREDIT_SUM,BU_sum_AMT_CREDIT_SUM_OVERDUE,BU_max_DAYS_CREDIT_UPDATE,BU_sum_AMT_ANNUITY,payment_day_missed_sum,payment_day_missed_mean,payment_day_missed_amt_mean,payment_day_missed_amt_max,payment_missed_sum,payment_missed_mean,payment_missed_amt_mean,payment_missed_amt_max,NUM_INSTALMENT_NUMBER_mean,SK_ID_PREV,ccb_MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,ccb_SK_DPD,ccb_SK_DPD_DEF,pct_balance_limit,pct_AMT_INST_TOTAL
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.

In [19]:
df_ccb.head()

,SK_ID_PREV,ccb_MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,ccb_SK_DPD,ccb_SK_DPD_DEF,pct_balance_limit,pct_AMT_INST_TOTAL
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,
100006,1489396.0,-3.5,0.000000,270000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000004,1.000000
100011,1843384.0,-38.5,54482.111149,164189.189189,2432.432432,2432.432432,0.0,0.0,3956.221849,4843.064189,4520.067568,52402.088919,54433.179122,54433.179122,0.054054,0.054054,0.0,0.0,25.767123,0.000000,0.000000,0.302683,12.004893
100013,2038692.0,-48.5,18159.919219,131718.750000,6350.000000,5953.125000,0.0,0.0,1454.539551,7168.346250,6817.172344,17255.559844,18101.079844,18101.079844,0.255556,0.239583,0.0,0.0,18.719101,0.010417,0.010417,0.115310,128.177348
100021,2594025.0,-10.0,0.000000,675000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000001,1.000000
100023,1499902.0,-7.5,0.000000,135000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000013,1.000000


In [20]:
df_pos.head()

,SK_ID_PREV,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF
SK_ID_CURR,,,,,,
100001,1.584045e+06,-72.555556,4.000000,1.444444,0.777778,0.777778
100002,1.038818e+06,-10.000000,24.000000,15.000000,0.000000,0.000000
100003,2.297665e+06,-43.785714,10.107143,5.785714,0.000000,0.000000
100004,1.564014e+06,-25.500000,3.750000,2.250000,0.000000,0.000000
100005,2.495675e+06,-20.000000,11.700000,7.200000,0.000000,0.000000


In [21]:
df_previous.head()

,SK_ID_PREV,previous_AMT_ANNUITY,AMT_APPLICATION,previous_AMT_CREDIT,AMT_DOWN_PAYMENT,previous_AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,SELLERPLACE_AREA,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,
100001,1369693.0,3951.000,24835.50,23787.00,2520.0,24835.5,13.0,1.0,0.104326,NaN,NaN,-1740.0,23.0,8.0,365243.0,-1709.0,-1499.0,-1619.0,-1612.0,0.0
100002,1038818.0,9251.775,179055.00,179055.00,0.0,179055.0,9.0,1.0,0.000000,NaN,NaN,-606.0,500.0,24.0,365243.0,-565.0,125.0,-25.0,-17.0,0.0
100003,2396755.0,64567.665,337500.00,348637.50,3442.5,337500.0,15.0,1.0,0.050030,NaN,NaN,-828.0,200.0,12.0,365243.0,-797.0,-647.0,-647.0,-639.0,1.0
100004,1564014.0,5357.250,24282.00,20106.00,4860.0,24282.0,5.0,1.0,0.212008,NaN,NaN,-815.0,30.0,4.0,365243.0,-784.0,-694.0,-724.0,-714.0,0.0
100005,2176837.0,4813.200,22308.75,20076.75,4464.0,44617.5,10.5,1.0,0.108964,NaN,NaN,-536.0,18.0,12.0,365243.0,-706.0,-376.0,-466.0,-460.0,0.0


In [22]:
# name = 'TARGET'
# print(application_train[name].value_counts().head())
# print(application_train[name].median())
# col_name(name)

## TARGET 

In [23]:
from credit_default import *
from helper.helper import *
from helper.feature_importance import *

In [50]:
applist2 = application_train.columns.tolist()

In [51]:
applist = ['TARGET','DAYS_BIRTH','REGION_RATING_CLIENT_W_CITY','AMT_CREDIT',
        'AMT_INCOME_TOTAL','DAYS_EMPLOYED','EXT_SOURCE_1','EXT_SOURCE_2',
        'EXT_SOURCE_3','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','WALLSMATERIAL_MODE',
        'CODE_GENDER','OCCUPATION_TYPE','FLAG_OWN_CAR','OWN_CAR_AGE',
        'AMT_ANNUITY','AMT_GOODS_PRICE',
        'DAYS_LAST_PHONE_CHANGE'] 


bureaulist =[
 'BU_sum_AMT_CREDIT_SUM',
 'BU_sum_AMT_CREDIT_SUM_OVERDUE',
 'BU_max_DAYS_CREDIT_UPDATE',
 'BU_sum_AMT_ANNUITY',
#  'BU_STATUS1_CNT',
#  'BU_STATUS2_CNT',
#  'BU_STATUS3_CNT',
#  'BU_STATUS4_CNT',
#  'BU_STATUS0_CNT',
#  'BU_STATUSC_CNT',
#  'BU_STATUSX_CNT'
]

previouslist = [
#  'previous_AMT_ANNUITY',
#  'AMT_APPLICATION',
#  'previous_AMT_CREDIT',
 'AMT_DOWN_PAYMENT',
#  'previous_AMT_GOODS_PRICE',
#  'HOUR_APPR_PROCESS_START',
#  'RATE_DOWN_PAYMENT',
#  'DAYS_DECISION',
 'SELLERPLACE_AREA',
 'CNT_PAYMENT',
#  'DAYS_FIRST_DRAWING',
 'DAYS_LAST_DUE_1ST_VERSION',
#  'DAYS_TERMINATION',
#  'NFLAG_INSURED_ON_APPROVAL'
]

poslist = [
 'MONTHS_BALANCE',
 'CNT_INSTALMENT',
 'CNT_INSTALMENT_FUTURE',
#  'SK_DPD',
 'SK_DPD_DEF']+['payment_day_missed_sum',
 'payment_day_missed_mean',
 'payment_day_missed_amt_mean',
#  'payment_day_missed_amt_max',
#  'payment_missed_sum',
 'payment_missed_mean',
 'payment_missed_amt_mean',
 'payment_missed_amt_max',
 'NUM_INSTALMENT_NUMBER_mean'
]

ccblist = [
#  'ccb_MONTHS_BALANCE',
#  'AMT_BALANCE',
#  'AMT_CREDIT_LIMIT_ACTUAL',
#  'AMT_DRAWINGS_ATM_CURRENT',
#  'AMT_DRAWINGS_CURRENT',
#  'AMT_DRAWINGS_OTHER_CURRENT',
#  'AMT_DRAWINGS_POS_CURRENT',
#  'AMT_INST_MIN_REGULARITY',
#  'AMT_PAYMENT_CURRENT',
#  'AMT_PAYMENT_TOTAL_CURRENT',
#  'AMT_RECEIVABLE_PRINCIPAL',
#  'AMT_RECIVABLE',
#  'AMT_TOTAL_RECEIVABLE',
 'CNT_DRAWINGS_ATM_CURRENT',
#  'CNT_DRAWINGS_CURRENT',
#  'CNT_DRAWINGS_OTHER_CURRENT',
 'CNT_DRAWINGS_POS_CURRENT',
#  'CNT_INSTALMENT_MATURE_CUM',
#  'ccb_SK_DPD',
#  'ccb_SK_DPD_DEF',
 'pct_balance_limit',
 'pct_AMT_INST_TOTAL'
]
preselect_cols = applist2 + bureaulist + previouslist + poslist + ccblist
print([item for item, count in collections.Counter(preselect_cols).items() if count > 1])
preselect_cols = list(set(preselect_cols))
list_to_drop = ['TARGET']

# list_to_drop = ['TARGET'] + list_to_drop2 

[]


In [ ]:
p = Pipeline([
    ('pre_select', PreselectColumns(preselect_cols)),  # Select columns to use in the training set
    ('clean_data', CleanData()), 
    ('add_nancol', BoolNan()),
    ('feature_engin', FeatureEnginner()),
    ('dummify_cat', Getdummies()),
    ("impute_nan", ReplaceNaN()),
    ('drop_cols', DropColumns(list_to_drop)),
#     ('rf', RandomForestClassifier(n_estimators=50)),
    ('adaboosting_model',AdaBoostClassifier(learning_rate=1, n_estimators=50)),
])

In [ ]:
y = df_train.TARGET
X = df_train.copy()

In [ ]:
scores = cross_val_score(p, X, y, scoring='roc_auc', cv=5, n_jobs=-1)
print(scores)
print(np.mean(scores))

```
[ 0.74106563  0.73974767  0.7373134   0.74421903  0.74564347]
0.741597839 - ada 50 trees rate 1

features:
['AMT_GOODS_PRICE',
 'EXT_SOURCE_3',
 'AMT_CREDIT',
 'DAYS_LAST_PHONE_CHANGE',
 'OWN_CAR_AGE',
 'EXT_SOURCE_1',
 'DAYS_BIRTH',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'REGION_RATING_CLIENT_W_CITY',
 'AMT_ANNUITY',
 'DAYS_EMPLOYED',
 'EXT_SOURCE_2',
 'Fe:amt_credit_income_ratio',
 'Fe:annuity_income_ratio',
 'OCCUPATION_TYPE_Core staff',
 'OCCUPATION_TYPE_Drivers',
 'OCCUPATION_TYPE_Low-skill Laborers',
 'NAME_INCOME_TYPE_Working',
 'WALLSMATERIAL_MODE_Panel',
 'NAME_EDUCATION_TYPE_Higher education',
 'NAME_EDUCATION_TYPE_Incomplete higher']
```

In [57]:
p = Pipeline([
    ('pre_select', PreselectColumns(preselect_cols)),  # Select columns to use in the training set
    ('clean_data', CleanData()), 
    ('add_nancol', BoolNan()),
    ('feature_engin', FeatureEnginner()),
    ('dummify_cat', Getdummies()),
    ("impute_nan", ReplaceNaN()),
    ('drop_cols', DropColumns(list_to_drop)),
])

y = df_train.TARGET
X = df_train.copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

mod = p.fit(X_train,y_train)
X_train_post = mod.transform(X_train)
X_test_post = mod.transform(X_test)

/Users/Alex/Documents/10_galvanize/galvanize/defaultrisk/credit_default.py:25: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  X = X.loc[:,self.preselect_cols]
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


['HOUR_APPR_PROCESS_START']
['HOUR_APPR_PROCESS_START']
['HOUR_APPR_PROCESS_START']


In [35]:
# rf = RandomForestClassifier(n_estimators=100)
# model = rf.fit(X_train_post, y_train)

In [58]:
ada = AdaBoostClassifier(learning_rate=1, n_estimators=100)
model = ada.fit(X_train_post, y_train)

In [59]:
y_predict_proba = model.predict_proba(X_test_post)[:,1:]
# y_predict = model.predict(X_test_post)
fpr, tpr, thresholds = roc_curve(y_test, y_predict_proba)
print(auc(fpr, tpr))

0.758908416965


`0.72295096362 - rf 100 trees
0.761663119848 - ada 100 trees 1 rate
`

In [ ]:
feature_importance = permutation_importance_auc(model, X_test_post.values, y_test)

In [ ]:
series_feature_importance = pd.Series(feature_importance,index=X_train_post.columns.tolist())
feature_importance = series_feature_importance.copy()
plot_feature_importance(feature_importance, show_n=20)

In [ ]:
list_to_drop2 = feature_importance.index[feature_importance<=0].tolist()
feature_importance.sort_values(ascending=False)

In [32]:
list_kept = feature_importance.index[feature_importance>0].tolist()
list_kept

['EXT_SOURCE_1',
 'pct_balance_limit',
 'DAYS_EMPLOYED',
 'BU_sum_AMT_CREDIT_SUM',
 'pct_AMT_INST_TOTAL',
 'AMT_GOODS_PRICE',
 'SELLERPLACE_AREA',
 'SK_DPD_DEF',
 'DAYS_LAST_PHONE_CHANGE',
 'AMT_ANNUITY',
 'payment_day_missed_sum',
 'NUM_INSTALMENT_NUMBER_mean',
 'BU_max_DAYS_CREDIT_UPDATE',
 'CODE_GENDER',
 'OWN_CAR_AGE',
 'REGION_RATING_CLIENT_W_CITY',
 'BU_sum_AMT_CREDIT_SUM_OVERDUE',
 'payment_day_missed_mean',
 'FLAG_OWN_CAR',
 'EXT_SOURCE_3',
 'AMT_CREDIT',
 'AMT_DOWN_PAYMENT',
 'payment_missed_amt_mean',
 'CNT_PAYMENT',
 'CNT_DRAWINGS_ATM_CURRENT',
 'CNT_DRAWINGS_POS_CURRENT',
 'DAYS_BIRTH',
 'DAYS_LAST_DUE_1ST_VERSION',
 'CNT_INSTALMENT',
 'EXT_SOURCE_2',
 'CNT_INSTALMENT_FUTURE',
 'BU_max_DAYS_CREDIT_UPDATE_nan',
 'Fe:amt_credit_income_ratio',
 'Fe:annuity_income_ratio',
 'Fe:mutiple_BU_credit_amt',
 'WALLSMATERIAL_MODE_Panel',
 'OCCUPATION_TYPE_Core staff',
 'OCCUPATION_TYPE_Medicine staff',
 'NAME_INCOME_TYPE_Commercial associate',
 'NAME_INCOME_TYPE_State servant',
 'NAME_E

In [ ]:
list_to_drop2

In [46]:
p = Pipeline([
    ('pre_select', PreselectColumns(preselect_cols)),  # Select columns to use in the training set
    ('clean_data', CleanData()), 
    ('add_nancol', BoolNan()),
    ('feature_engin', FeatureEnginner()),
    ('dummify_cat', Getdummies()),
    ("impute_nan", ReplaceNaN()),
    ('drop_cols', DropColumns(list_to_drop)),
#     ('rf', RandomForestClassifier(n_estimators=50)),
    ('adaboosting_model',AdaBoostClassifier(learning_rate=0.8, n_estimators=2000)),
])
y = df_train.TARGET
X = df_train.copy()
model = p.fit(X,y)


In [47]:
probabilities = model.predict_proba(df_test)[:, 1]
submit = application_test[['SK_ID_CURR']]
submit['TARGET'] = probabilities
submit.head()

/Users/Alex/Documents/10_galvanize/galvanize/defaultrisk/credit_default.py:25: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  X = X.loc[:,self.preselect_cols]
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

,SK_ID_CURR,TARGET
0,100001,0.499567
1,100005,0.499675
2,100013,0.499370
3,100028,0.499419
4,100038,0.499684


In [48]:
submit.to_csv('abamodel4.csv', index = False)

In [ ]:
def add_diff_per_cat(cat_col,num_col,df):
    new_df = df.copy()
    mean_df = new_df.groupby(cat_col)[[num_col]].mean()
    new_df = new_df.join(mean_df, on=cat_col,
                    how='left', rsuffix='_per_' + cat_col)
    new_df['Fe:diff_'+new_df.columns[-1]] = new_df[num_col] - new_df[num_col+'_per_'+cat_col]
#     new_df.drop(num_col+'_per_'+cat_col, axis=1, inplace=True)
    return new_df

In [ ]:
temp = add_diff_per_cat('OCCUPATION_TYPE','AMT_ANNUITY',app)

In [ ]:
temp.head()

In [ ]:
cols = ['TARGET','DAYS_BIRTH','REGION_RATING_CLIENT_W_CITY','AMT_CREDIT',
        'AMT_INCOME_TOTAL','DAYS_EMPLOYED','EXT_SOURCE_1','EXT_SOURCE_2',
        'EXT_SOURCE_3','OBS_60_CNT_SOCIAL_CIRCLE', 'FLAG_OWN_REALTY',
        'NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','WALLSMATERIAL_MODE',
        'CODE_GENDER','OCCUPATION_TYPE','FLAG_OWN_CAR','OWN_CAR_AGE',
        'AMT_ANNUITY','AMT_GOODS_PRICE','FLAG_PHONE','FLAG_EMP_PHONE',
        'FLAG_CONT_MOBILE','FLAG_MOBIL','DAYS_LAST_PHONE_CHANGE']

cols = list(set(cols))
# cols += docs

def features_engineer(df, col_list):
    newdf = df[col_list].copy()

    
    # Loan
    newdf['AMT_CREDIT'] = np.log(newdf['AMT_CREDIT'])
    newdf['AMT_ANNUITY'] = np.log(newdf['AMT_ANNUITY'])
    newdf['AMT_GOODS_PRICE'] = np.log(newdf['AMT_GOODS_PRICE'])
    
    # About Age
    newdf["DAYS_BIRTH"] = newdf["DAYS_BIRTH"]/-365
    newdf.loc[newdf["DAYS_EMPLOYED"]>0,'DAYS_EMPLOYED']=1
    
    # binarize income types
    dummies = pd.get_dummies(newdf["NAME_INCOME_TYPE"])
    if 'Maternity leave' in dummies.columns:
        dummies.drop('Maternity leave', axis=1, inplace=True)
    newdf = pd.concat([newdf, dummies], axis=1)
    newdf.drop('NAME_INCOME_TYPE', axis=1, inplace=True)
    
    # Education types
    dummies = pd.get_dummies(newdf["NAME_EDUCATION_TYPE"])
    if 'Academic degree' in dummies.columns:
        dummies.drop('Academic degree', axis=1, inplace=True)
    newdf = pd.concat([newdf, dummies], axis=1)
    newdf.drop('NAME_EDUCATION_TYPE', axis=1, inplace=True)    
    
    # Asset - house
    newdf['FLAG_OWN_REALTY'] = pd.get_dummies(newdf['FLAG_OWN_REALTY'],drop_first=True)
    dummies = pd.get_dummies(newdf["WALLSMATERIAL_MODE"])
    dummies.drop(['Block','Others','Mixed'], axis=1, inplace=True)
    newdf = pd.concat([newdf, dummies], axis=1)
    newdf.drop('WALLSMATERIAL_MODE', axis=1, inplace=True) 
    
    
    # Asset - car
    newdf['FLAG_OWN_CAR'] = pd.get_dummies(newdf['FLAG_OWN_CAR'],drop_first=True)
    newdf['OWN_CAR_AGE_squared'] = newdf['OWN_CAR_AGE']
    newdf['FLAG_OWN_CAR_CAR_AGE']=newdf['FLAG_OWN_CAR']*newdf['OWN_CAR_AGE']
    
    # Phone
    newdf['DAYS_LAST_PHONE_CHANGE']=newdf['DAYS_LAST_PHONE_CHANGE']/-365
    
    # Gender
    dummies = pd.get_dummies(newdf["CODE_GENDER"])
    if 'XNA' in dummies.columns:      
        dummies.drop('XNA', axis=1, inplace=True)
    newdf.drop('CODE_GENDER', axis=1, inplace=True)
    
    # Job types
    dummies = pd.get_dummies(newdf["OCCUPATION_TYPE"])
    newdf = pd.concat([newdf, dummies[jobs]], axis=1)
    newdf.drop('OCCUPATION_TYPE', axis=1, inplace=True) 
    
    # Income 
    newdf['log_income_total'] = np.log(newdf['AMT_INCOME_TOTAL'])
    newdf['log_income_cut']=pd.cut(
    newdf['log_income_total'], bins=[0,11.503,11.899,12.218,18.579],
    labels=["low", "medium-low", "medium", "high"])
    dummies = pd.get_dummies(newdf["log_income_cut"])
    dummies.drop(['low','medium'], axis=1, inplace=True)
    newdf.drop(['log_income_cut','log_income_total'], axis=1, inplace=True) 
    
    # Repayment mutiples
    newdf['loan_income_mutiple'] = newdf['AMT_CREDIT']/newdf['AMT_INCOME_TOTAL']
    newdf['annuity_income'] = newdf['AMT_ANNUITY']/newdf["AMT_INCOME_TOTAL"]
    newdf['goods_price_income'] = newdf['AMT_GOODS_PRICE']/newdf["AMT_INCOME_TOTAL"]
    
    # Documents
#     newdf['total_doc'] = 0
#     for doc in docs:
#         newdf['total_doc'] += newdf[doc]
#         newdf.drop(doc, axis=1, inplace=True)
    
    # Some interaction terms
    newdf['Days_vs_source_1'] = newdf['EXT_SOURCE_1']/newdf['DAYS_BIRTH']
    newdf['Days_vs_employed'] = newdf['DAYS_EMPLOYED']*newdf['DAYS_BIRTH']
    newdf['EXT_SOURCE_2_vs_REGION_RATING_CLIENT_W_CITY'] = newdf['EXT_SOURCE_2']*newdf['REGION_RATING_CLIENT_W_CITY']
    
    return newdf

In [ ]:
df_train_2 = features_engineer(application_train,cols)

if 'TARGET' in cols:
    cols_no_target = cols.copy()
    cols_no_target.remove('TARGET')
else:
    cols_no_target=cols.copy()
    
df_test = features_engineer(application_test,cols_no_target)

In [ ]:
X = df_train_2.drop('TARGET',axis=1).values
y = df_train_2['TARGET'].values


X_train, X_test, y_train, y_test = train_test_split(X, y)

# imputer all the nan by median or mean
imp = Imputer(strategy='median') 
imp.fit(X_train) 

# transform the test & train data
X_train=imp.transform(X_train)
X_test=imp.transform(X_test)

# rescaling features is important
# scaler = MinMaxScaler(feature_range = (0, 1))
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

print('Training data shape: ', X_train.shape)
print('Testing data shape: ', X_test.shape)

In [ ]:
kf = KFold(n_splits=5)
kf.get_n_splits(X_train)

accuracies = []
precisions = []
recalls = []
aucs = []

for train_index, test_index in kf.split(X_train):
    model = LogisticRegression(class_weight='balanced')
    model.fit(X_train[train_index], y_train[train_index])
    
#     proba = model.predict_proba(X_train[test_index])[:, 1]
#     y_predict = proba>=0.15   
#     changing the thresholds can change the tpr and fpr
    
    y_predict = model.predict(X_train[test_index])
    y_true = y_train[test_index]
    fpr, tpr, thresholds = roc_curve(y_true, y_predict)
    
    accuracies.append(accuracy_score(y_true, y_predict))
    precisions.append(precision_score(y_true, y_predict))
    recalls.append(recall_score(y_true, y_predict))
    aucs.append(auc(fpr, tpr))
    
    
print("accuracy:", np.average(accuracies))
print("precision:", np.average(precisions))
print("recall:", np.average(recalls))
print("auc:", np.average(aucs))

to compare:
```
old model:
accuracy: 0.687048266413
precision: 0.159131279008
recall: 0.670711590906
auc: 0.679588691017
```


In [ ]:
model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)
probabilities = model.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, probabilities)
print(auc(fpr, tpr))# old:0.744496536485
plt.plot(fpr, tpr)
plt.xlabel("False Positive Rate (1 - Specificity)")
plt.ylabel("True Positive Rate (Sensitivity, Recall)")
plt.title("ROC plot")
plt.show();

In [ ]:
model = RandomForestClassifier(n_estimators=500)
model.fit(X_train, y_train)
probabilities = model.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, probabilities)
print(auc(fpr, tpr))# old:0.744496536485
plt.plot(fpr, tpr)
plt.xlabel("False Positive Rate (1 - Specificity)")
plt.ylabel("True Positive Rate (Sensitivity, Recall)")
plt.title("ROC plot")
plt.show();

In [ ]:
# predictors = list(df_train_2.columns)
# predictors.remove('TARGET')
# # call forward_step_select
# mods = forward_step_select(df,predictors,['TARGET'])

## Submission

In [ ]:
application_train = pd.read_csv('data/application_train.csv.zip') 
application_test = pd.read_csv('data/application_test.csv.zip') 

Last step, retrain model with all the data.

top idea:
obs_30 or obs_60
Amt_credit / income
Fam size vs cnt chidren
own realty
own car
nice to have:
amt_req_redit_buro
housing type

In [ ]:
df_train = features_engineer(df=application_train,col_list=cols)

if 'TARGET' in cols:
    cols_no_target = cols.copy()
    cols_no_target.remove('TARGET')
else:
    cols_no_target=cols.copy()
    
df_test = features_engineer(df=application_test,col_list=cols_no_target)

In [ ]:
print(df_train.shape)
print(df_test.shape)

In [ ]:
X_train = df_train.drop('TARGET',axis=1).values
y_train = df_train['TARGET'].values
X_test = df_test.values

In [ ]:
imp = Imputer(strategy='median') 
imp.fit(X_train) 

# transform the test & train data
X_train=imp.transform(X_train)
X_test=imp.transform(X_test)


scaler = MinMaxScaler(feature_range = (0, 1))
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print('Training data shape: ', X_train.shape)
print('Testing data shape: ', X_test.shape)

In [ ]:
model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)
probabilities = model.predict_proba(X_test)[:, 1]

In [ ]:
submit = application_test[['SK_ID_CURR']]
submit['TARGET'] = probabilities
submit.head()

In [ ]:
submit.to_csv('logistic_baseline5.csv', index = False)

Your submission scored 0.733, which is an improvement of your previous score of 0.726. 

top idea:
obs_30 or obs_60
Amt_credit / income
Fam size vs cnt chidren
own realty
own car
nice to have:
amt_req_redit_buro
housing type
